# Part 10. 그룹화와 요약 테이블 만들기

## 1) 원본 로그와 요약 테이블의 차이
원본 로그는 보통:
- 주문 1건
- 방문 1회
- 클릭 1번

같은 사건 단위로 쌓인다.

하지만 분석에서 원하는 건:
- 날짜별 매출
- 매장별 매출
- 메뉴별 매출

분석은 원본 로그를 그대로 읽는 게 아니라, 요약 테이블을 통해 패턴을 비교하는 과정이다. <br>
요약 테이블은 분석 질문에 답하기 위한 형태다.

## 2) GroupBy의 역할
GroupBy의 역할은 한 문장으로 정리할 수 있다.
- 기준을 나누고, 각 그룹을 집계한다.

여기서 핵심은 두 가지다.
- 기준이 무엇인가? (date / store / menu 등)
- 무엇을 집계할 것인가? (sales 합계 / 평균 등)


GroupBy는 요약 테이블을 만드는 가장 기본 도구다.

## 3) GroupBy 문장 구조 (기본 문법)
```python
# 단일 기준 + 단일 컬럼 집계
df.groupby(기준)[값컬럼].집계함수()

# 다중 기준 + 단일 컬럼 집계
df.groupby([기준1, 기준2])[값컬럼].집계함수()

# 기준으로 나눈 뒤 여러 집계를 한 번에
df.groupby(기준).agg({...})

```

### 짚고넘어가기: 기간 단위 기준 만들기: to_period
```python
df["date"].dt.to_period("M")

```
- 날짜를 일정한 기간 단위(월, 분기 등)로 묶기 위한 변환
    - "M": 월 단위
    - "Q": 분기 단위
    - "Y": 연 단위

In [7]:
import pandas as pd

df = pd.DataFrame([
    {"date": "2026-01-01", "store": "광교점", "menu": "Americano", "sales": 9000},
    {"date": "2026-01-01", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-02", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-03", "store": "수원점", "menu": "Americano", "sales": 4500},
    {"date": "2026-01-03", "store": "수원점", "menu": "Mocha",     "sales": 5500},
])

df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)

store_report = df.groupby("store")["sales"].sum().reset_index(name="total_sales")
menu_report = df.groupby("menu")["sales"].mean().reset_index(name="avg")


print("원본 로그(주문 1건 단위):")
print(df)

print("\n매장 단위 요약(매장별 매출 합계):")
print(store_report)

print("\n메뉴 단위 요약(메뉴별 매출 합계):")
print(menu_report)

원본 로그(주문 1건 단위):
        date store       menu  sales       ym
0 2026-01-01   광교점  Americano   9000  2026-01
1 2026-01-01   광교점      Latte   5000  2026-01
2 2026-01-02   광교점      Latte   5000  2026-01
3 2026-01-03   수원점  Americano   4500  2026-01
4 2026-01-03   수원점      Mocha   5500  2026-01

매장 단위 요약(매장별 매출 합계):
  store  total_sales
0   광교점        19000
1   수원점        10000

메뉴 단위 요약(메뉴별 매출 합계):
        menu     avg
0  Americano  6750.0
1      Latte  5000.0
2      Mocha  5500.0


## 4) 집계 함수(aggregation) 핵심 5종
실무에서 사용하는 집계 함수는 생각보다 종류가 많지 않다.<br>
아래 다섯 가지만 제대로 쓰면 대부분의 요약 테이블을 만들 수 있다.

- sum: 총합 (총매출, 총수량)
- mean: 평균 (평균 매출, 평균 수량, 성공률)
- count: 개수 (주문 건수 = 행 개수)
- nunique: 고유값 개수 (고유 메뉴 수, 고유 고객 수)
- min / max: 최소/최대 (최소/최대 수량, 최대 매출 등)

중요한 건 "함수를 많이 아는 것"이 아니라 지금 어떤 질문을 하고 있는가다.

### agg란?
GroupBy 결과에 집계 함수 하나만 쓰는 경우는 드물다.<br>
대부분은
- 합계도 보고 싶고
- 평균도 보고 싶고
- 개수도 같이 보고 싶다.

이럴 때 필요한 게 agg다.

여러 집계를 한 번에, 명시적으로 관리할 수 있어 요약 테이블의 구조가 예측 가능해지고, 이후 후처리도 훨씬 쉬워진다.

In [8]:
import pandas as pd

df = pd.DataFrame([
    {"store": "광교점", "menu": "Americano", "qty": 2, "sales": 9000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "qty": 1, "sales": 5000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "qty": 2, "sales": 0,    "paid": False},
    {"store": "수원점", "menu": "Americano", "qty": 1, "sales": 4500, "paid": True},
    {"store": "수원점", "menu": "Mocha",     "qty": 1, "sales": 0,    "paid": False},
])

summary = df.groupby("store").agg(
    total_sales=("sales", "sum"),
    avg_sales=("sales", "mean"),
    orders=("menu", "count"),
    unique_menus=("menu", "nunique"),
    min_qty=("qty", "min"),
    max_qty=("qty", "max"),
    paid_rate=("paid", "mean"),
).reset_index()

print("사용한 집계 함수 출력")
print(summary)

사용한 집계 함수 출력
  store  total_sales    avg_sales  orders  unique_menus  min_qty  max_qty  \
0   광교점        14000  4666.666667       3             2        1        2   
1   수원점         4500  2250.000000       2             2        1        1   

   paid_rate  
0   0.666667  
1   0.500000  


## 4) MultiIndex가 뭐고 왜 불편한가?
GroupBy 결과는 종종 인덱스가 여러 단계로 생긴다. (MultiIndex) <br>
- 컬럼 접근이 복잡해진다
- 후처리(정렬, 병합, 피벗)가 불편해진다

그래서 요약 테이블을 만들 때는 보통:
- reset_index()로 인덱스를 컬럼으로 되돌리거나
- 이후 피벗으로 넓은 형태로 바꾸는 흐름으로 간다

In [9]:
df.groupby("menu")["sales"].agg(["sum", "mean", "count"])
# 이렇게 되면 컬럼 접근이 불편해지고 rename / merge / 시각화하기 어렵다

,sum,mean,count
menu,,,
Americano,13500,6750.0,2
Latte,5000,2500.0,2
Mocha,0,0.0,1


In [10]:
summary = df.groupby("store").agg(
    total_sales=("sales", "sum"),
    avg_sales=("sales", "mean"),
    orders=("menu", "count"),
).reset_index()
summary
# 그래서 실무에서는 이런식으로 컬럼명을 직접 지정하는 agg 형태를 더 선호한다.

,store,total_sales,avg_sales,orders
0,광교점,14000,4666.666667,3
1,수원점,4500,2250.000000,2


## 5) 피벗(표 형태 변환) 이론

GroupBy 결과는 보통 이런 형태다.
- 기준 컬럼이 여러 개
- 행이 길게 늘어짐
- 비교하려면 눈으로 계산해야 함

이 상태는 계산에는 적합하지만 보고서·시각화에는 불편하다.

이때 필요한 게 피벗이다.<br>
피벗은 데이터를 계산하기 위해 쓰는 도구가 아닌, 결과를 정리하기 위해 쓰는 도구다.

In [11]:
import pandas as pd

# 예시 데이터(일부 조합이 반복되도록 일부러 2행 넣음: 2026-01 + Mon)
df = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "sales": 100},
    {"ym": "2026-01", "day": "Mon", "sales": 50},   # 같은 조합이 2개라서 pivot이 바로 안 됨
    {"ym": "2026-01", "day": "Tue", "sales": 80},
    {"ym": "2026-02", "day": "Mon", "sales": 120},
    # 2026-02의 Tue가 없어서 피벗 후 NaN이 생길 수 있음
])

print("원본(long 형태)")
df
# 이 상태에서 바로 매장 비교도 어렵고, 메뉴별 비교도 한눈에 볼수 없다.

원본(long 형태)


,ym,day,sales
0,2026-01,Mon,100
1,2026-01,Mon,50
2,2026-01,Tue,80
3,2026-02,Mon,120


### 긴 형태(long) → 넓은 형태(wide)
피벗을 이해하려면 데이터의 "형태"부터 이해해야 한다.

#### 긴 형태 (long)
- 기준 컬럼이 여러 개
- 행이 많고 세로로 길다
- GroupBy 결과가 대표적이다

특징:
- 계산·집계에 유리
- 비교에는 불리

대부분의 원본 데이터와 GroupBy 결과는 long 형태다.

#### 넓은 형태 (wide)
- 하나의 기준이 행
- 다른 기준이 열
- 값이 셀에 들어간다

특징:
- 비교·보고에 유리
- 시각화에 바로 사용 가능

피벗의 역할은 long 형태를 wide 형태로 바꾸는 것이다.

```python
데이터프레임.pivot(
    index="행 기준",
    columns="열 기준",
    values="값"
)
```

In [12]:
# 1단계: 먼저 요약 (칸당 값 1개 만들기)
summary = (
    df.groupby(["ym", "day"])["sales"]
    .sum()
    .reset_index()
)

print("\nGroupBy 요약 결과")
print(summary)

# 2단계: pivot으로 표 설계
report = summary.pivot(
    index="ym",
    columns="day",
    values="sales"
)

print("\npivot 결과")
print(report)

# 즉, pivot은 표 설계 도구다
# "이걸 행으로, 이걸 열로, 이 값을 채운다"라고 말하는 방식이다


GroupBy 요약 결과
        ym  day  sales
0  2026-01  Mon    150
1  2026-01  Tue     80
2  2026-02  Mon    120

pivot 결과
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN


## 6) pivot vs unstack은 뭐가 다른가?
pivot과 unstack은 결과는 비슷해 보이지만 출발점이 다르다.

pivot
- DataFrame 상태에서 사용
- index / columns / values를 직접 지정
- 이미 집계가 끝난 데이터에 적합

요약된 테이블을 다시 배치하는 용도

<br>

unstack
- GroupBy 결과에서 사용
- 인덱스 레벨을 열로 올린다
- MultiIndex를 다룰 때 등장한다

GroupBy 결과를 표로 펼치는 용도

<br>

무엇을 쓰느냐 보단 지금 데이터가<br>
DataFrame인가, GroupBy 결과 인가가 더 중요하다.

In [13]:
# GroupBy 결과 (MultiIndex Series)
multi = df.groupby(["ym", "day"])["sales"].sum()

print("\nGroupBy 결과 (MultiIndex)")
print(multi)

# unstack으로 인덱스 레벨을 열로 펼침
wide = multi.unstack()

print("\nunstack 결과")
print(wide)
# 즉, unstack은 이미 만들어진 구조를 펼치는 도구다


GroupBy 결과 (MultiIndex)
ym       day
2026-01  Mon    150
         Tue     80
2026-02  Mon    120
Name: sales, dtype: int64

unstack 결과
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN


## 7) 피벗에서 자주하는 실수 포인트

In [14]:
import pandas as pd

df = pd.DataFrame([
    {"date": "2026-01-01", "menu": "Americano", "store": "광교점", "sales": 9000},
    {"date": "2026-01-02", "menu": "Latte",     "store": "광교점", "sales": 5000},
    {"date": "2026-01-03", "menu": "Latte",     "store": "수원점", "sales": 5000},
    {"date": "2026-01-06", "menu": "Americano", "store": "광교점", "sales": 4500},
    {"date": "2026-02-03", "menu": "Mocha",     "store": "수원점", "sales": 5500},
])

df["date"] = pd.to_datetime(df["date"])
df

,date,menu,store,sales
0,2026-01-01,Americano,광교점,9000
1,2026-01-02,Latte,광교점,5000
2,2026-01-03,Latte,수원점,5000
3,2026-01-06,Americano,광교점,4500
4,2026-02-03,Mocha,수원점,5500


### ① 값이 여러 개라서 피벗이 안 되는 경우
같은 (index, columns) 조합에 값이 여러 개 있으면 pivot은 동작하지 않는다.
- 아직 집계가 안 됐거나
- 집계 기준이 부족한 상태

해결법은 2가지가 있다.
1. GroupBy로 집계한다. (groupby + unstack)
2. pivot_table 을 사용한다.


In [15]:
menu_store_sales = (
    df.groupby(["menu", "store"])["sales"]
    .sum()
    .reset_index()
)

# GroupBy 피벗
menu_store_sales.pivot(
    index="menu",
    columns="store",
    values="sales"
)

print("GroupBy를 한 피벗")
menu_store_sales

# pivot_table을 사용한 피벗
report = df.pivot_table(
    index="menu",
    columns="store",
    values="sales",
    aggfunc="sum"
)

print("\npivot_table을 사용한 피벗")
report

GroupBy를 한 피벗

pivot_table을 사용한 피벗


store,광교점,수원점
menu,,
Americano,13500.0,NaN
Latte,5000.0,5000.0
Mocha,NaN,5500.0


### ② 빈 칸(NaN)이 생기는 것
피벗 결과에 NaN이 생긴다는 건 해당 조합의 데이터가 없다는 뜻이다.

이건 오류가 아니라:
- 데이터의 구조적 특성
- 혹은 매핑 누락 신호다

분석 전에:
- 그대로 둘지
- 0으로 채울지
- 제외할지

의미를 먼저 판단해야 한다.

In [16]:
df

,date,menu,store,sales
0,2026-01-01,Americano,광교점,9000
1,2026-01-02,Latte,광교점,5000
2,2026-01-03,Latte,수원점,5000
3,2026-01-06,Americano,광교점,4500
4,2026-02-03,Mocha,수원점,5500


In [17]:
# GroupBy 피벗
pivot_result = df.pivot_table(
    index="menu",
    columns="store",
    values="sales",
    aggfunc="sum"
)

print("GroupBy를 한 피벗.fillna")
pivot_result.fillna(0)


# 보강: 날짜 × 매장 피벗 (리포트용)
date_store_sales = (
    df.groupby(["date", "store"], )["sales"]
    .sum()
    .reset_index()
)

date_store_sales.pivot(    
    index="date",
    columns="store",
    values="sales").fillna(0)

df.pivot_table(
    index="date",
    columns="store",
    values="sales",
    aggfunc="sum"
).fillna(0)

GroupBy를 한 피벗.fillna


store,광교점,수원점
date,,
2026-01-01,9000.0,0.0
2026-01-02,5000.0,0.0
2026-01-03,0.0,5000.0
2026-01-06,4500.0,0.0
2026-02-03,0.0,5500.0
